In [1]:
import pandas as pd#красивые таблички

TODO NEXT поколениям: не сделано красивое описание строчки для случая, когда r == '0'*8

In [2]:
import re

In [3]:
def indexing(matched):
    return '<sub>'+matched[0][1:]+'</sub>'
def overline(matched):
    return '<SPAN STYLE="text-decoration:overline">'+matched[0][2:-2]+'</SPAN>'

In [4]:
#вывод табличек в строках которых есть символ \n
from IPython.display import display, HTML
def pretty_print(df):
    a = df.to_html().replace("\\n","<br>")
    a = re.sub('--(\w+)--',overline,a)
    a = re.sub('_\d+',indexing,a)
    return display( HTML( a ) )

In [5]:
len_o = 8

2 вариант

In [6]:
A = 1318; B = 19

In [7]:
def dopCode(n,s=1):
    return 2**(len_o*s)-n

In [8]:
def bin_sum(a,b):
    a = list(map(int,list(a)))
    b = list(map(int,list(b)))
    d = len(a)-len(b)
    if d > 0:
        b = [0]*d+b
    if d < 0:
        a = [0]*abs(d)+a
    c=0
    s=''
    for i,j in zip(a[::-1],b[::-1]):
        s = s + chr((i+j+c)%2+ord('0'))
        c = (i+j+c)//2
    s=s[::-1]
    return s,c

In [9]:
bin_sum('1101','10101')

('00010', 1)

1. Умножение в дополнительных кодах с применением коррекции.

In [33]:
def div(A,B):
    AInDopcode=False
    if A < 0:
        A=dopCode(-A,2)
        AInDopcode=True
    binA = bin(A)[2:]
    binA = ('1' if AInDopcode else '0')*(len_o*2-len(binA))+binA
    
    BInDopcode=False
    if B < 0:
        B=dopCode(-B,1)
        BInDopcode=True
    binB = bin(B)[2:]
    binB = ('1' if BInDopcode else '0')*(len_o*2-len(binB))+binB
    
    
    binBdop = bin(dopCode(B,1))[2:]
    binBdop = ('0' if BInDopcode else '1')*(len_o*2-len(binBdop))+binBdop
    df = pd.DataFrame([],columns=['Операнды и действия','Делимое и остаток (старшие разряды)',
                              'Делимое и остаток (младшие разряды), частное','Пояснения'])
    k = ['','','','']
    bp = 'доп' if BInDopcode else 'пр'
    bpdop = 'пр' if BInDopcode else 'доп'
    if binA[0]!=binB[0]:
        r = bin_sum(binA,binB)[0]
        ans = '1' if r[0] == binB[0] else '0'
        k[0] = '[B]'+bp+'\n'+\
                "R'_1\n"
        k[1] = binB[:len_o]+'\n'+r[:len_o]+'\n'
        k[2] = binB[len_o:]+'\n'+r[len_o:]+'\n'
        k[3] = 'Сложение с делителем, выровненным по младшим разрядам\n'
    else:
        r = binA
        ans='1'
    binB = binB[len_o:]
    binBdop = binBdop[len_o:]
    step = 1
    while(step != len_o+1):
        r = r[1:]
        k[3] += 'Сдвиг остатка влево\n'
        k[0] += "--R--"+("'" if step == 1 else "")+"_"+str(step)+'\n+\n['+('-' if ans[-1] else '')+'B]'+(bpdop if ans[-1] else bp)+'\n'
        k[0] += 'R_'+str(step)
        curB = binBdop if ans[-1]=='1' else binB
        if ans[-1] == '1':
            k[3] += 'Вычитание делителя'
        else:
            k[3] += 'Сложение с делителем'
        k[3]+='\n'*2
        k[1] += r[:len_o]+'\n\n'+curB+'\n'
        #print(r[:len_o],curB)
        r = bin_sum(r[:len_o],curB)[0] + r[len_o:]
        k[1] += r[:len_o]
        if step == 1:
            if r[0] == binA[0]:
                return 'eror'
            ans=''
            k[3] += 'Знак первого остатка не совпадает со знаком делимого–деление корректно\nФормирование знака частного'
        ans = ans + ('1' if r[0] == binB[0] else '0')
        k[2] += (r[len_o:]+'|' if step != len_o else '')+ans[:-1]+'0\n\n\n'
        k[2] += (r[len_o:]+'|' if step != len_o else '')+ans
        if step != 1:
            k[3]+='Формирование цифры частного'
        step+=1
        df=pd.concat((df,pd.DataFrame([k],columns=['Операнды и действия','Делимое и остаток (старшие разряды)',
                              'Делимое и остаток (младшие разряды), частное','Пояснения'])),axis=0,ignore_index=True)
        k=['','','','']
    k = ['[A]'+('доп' if AInDopcode else 'пр'),binA[:len_o],binA[len_o:],'Делимое']
    df=pd.concat((pd.DataFrame([k],columns=['Операнды и действия','Делимое и остаток (старшие разряды)',
                              'Делимое и остаток (младшие разряды), частное','Пояснения']),df),axis=0,ignore_index=True)
    if r == '0'*len_o and ans[-1]=='0':
        df[-1,2] = df[-1,2][:-1]+'1'
    if r[0] != binA[0]:
        curB = binBdop if ans[-1]=='1' else binB
        r = bin_sum(r[:len_o],curB)[0] + r[len_o:]
        k = ['[B]'+('пр' if BInDopcode else 'доп')+'\nR_6',curB+'\n'+r,'\n'+ans,'Коррекция остатка: '+\
             ('сложение с делителем' if (ans[-1]=='0') else 'вычитание делителя')+'\nРезультат']
        df=pd.concat((df,pd.DataFrame([k],columns=['Операнды и действия','Делимое и остаток (старшие разряды)',
                              'Делимое и остаток (младшие разряды), частное','Пояснения'])),axis=0,ignore_index=True)
    return df

In [34]:
pretty_print(div(A,B))

,Операнды и действия,Делимое и остаток (старшие разряды),"Делимое и остаток (младшие разряды), частное",Пояснения
0,[A]пр,00000101,00100110,Делимое
1,R'1+[-B]допR1,000010101110110111110111,0100110|00100110|0,Сдвиг остатка влевоВычитание делителяЗнак первого остатка не совпадает со знаком делимого–деление корректноФормирование знака частного
2,R2+[-B]допR2,111011100001001100000001,100110|00100110|01,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
3,R3+[-B]допR3,000000111110110111110000,00110|01000110|010,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
4,R4+[-B]допR4,111000000001001111110011,0110|01000110|0100,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
5,R5+[-B]допR5,111001100001001111111001,110|01000110|01000,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
6,R6+[-B]допR6,111100110001001100000110,10|01000010|010001,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
7,R7+[-B]допR7,000011011110110111111010,0|01000100|0100010,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
8,R8+[-B]допR8,111101000001001100000111,0100010001000101,Сдвиг остатка влевоСложение с делителемФормирование цифры частного


In [35]:
pretty_print(div(A,-B))

,Операнды и действия,Делимое и остаток (старшие разряды),"Делимое и остаток (младшие разряды), частное",Пояснения
0,[A]пр,00000101,00100110,Делимое
1,[B]допR'1R'1+[-B]прR1,1111111100000101000010101110110111110111,11101101000100110010011|00010011|1,"Сложение с делителем, выровненным по младшим разрядамСдвиг остатка влевоСложение с делителемЗнак первого остатка не совпадает со знаком делимого–деление корректноФормирование знака частного"
2,R2+[-B]прR2,111011100001001100000001,010011|10010011|10,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
3,R3+[-B]прR3,000000101110110111101111,10011|10010011|101,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
4,R4+[-B]прR4,110111110001001111110010,0011|10100011|1011,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
5,R5+[-B]прR5,111001000001001111110111,011|10110011|10111,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
6,R6+[-B]прR6,111011100001001100000001,11|10111011|101110,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
7,R7+[-B]прR7,000000111110110111110000,1|10111001|1011101,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
8,R8+[-B]прR8,111000010001001111110100,1011101010111011,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
9,[B]прR6,0001001100000111,10111011,Коррекция остатка: вычитание делителяРезультат


In [36]:
pretty_print(div(-A,B))

,Операнды и действия,Делимое и остаток (старшие разряды),"Делимое и остаток (младшие разряды), частное",Пояснения
0,[A]доп,11111010,11011010,Делимое
1,[B]прR'1R'1+[-B]допR1,0000000011111010111101010001001100001000,00010011111011011101101|01101101|1,"Сложение с делителем, выровненным по младшим разрядамСдвиг остатка влевоСложение с делителемЗнак первого остатка не совпадает со знаком делимого–деление корректноФормирование знака частного"
2,R2+[-B]допR2,000100011110110111111110,101101|10101101|10,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
3,R3+[-B]допR3,111111010001001100010000,01101|10001101|101,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
4,R4+[-B]допR4,001000001110110100001101,1101|10101101|1011,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
5,R5+[-B]допR5,000110111110110100001000,101|10110101|10111,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
6,R6+[-B]допR6,000100011110110111111110,01|10111001|101110,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
7,R7+[-B]допR7,111111000001001100001111,1|10111001|1011101,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
8,R8+[-B]допR8,000111111110110100001100,1011101010111011,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
9,[B]допR6,1110110111111001,10111011,Коррекция остатка: вычитание делителяРезультат


In [37]:
pretty_print(div(-A,-B))

,Операнды и действия,Делимое и остаток (старшие разряды),"Делимое и остаток (младшие разряды), частное",Пояснения
0,[A]доп,11111010,11011010,Делимое
1,R'1+[-B]прR1,111101010001001100001000,1011010|01011010|0,Сдвиг остатка влевоВычитание делителяЗнак первого остатка не совпадает со знаком делимого–деление корректноФормирование знака частного
2,R2+[-B]прR2,000100011110110111111110,011010|00011010|01,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
3,R3+[-B]прR3,111111000001001100001111,11010|01011010|010,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
4,R4+[-B]прR4,000111111110110100001100,1010|01001010|0100,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
5,R5+[-B]прR5,000110011110110100000110,010|01000010|01000,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
6,R6+[-B]прR6,000011001110110111111001,10|01000010|010001,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
7,R7+[-B]прR7,111100110001001100000110,0|01000100|0100010,Сдвиг остатка влевоВычитание делителяФормирование цифры частного
8,R8+[-B]прR8,000011001110110111111001,0100010001000101,Сдвиг остатка влевоСложение с делителемФормирование цифры частного
